# Simple Shortcut API Client

This Jupyter notebook provides example usage of this simple Python Shortcut API client.

## Getting Started

In [53]:
from scapi import ShortcutClient

In [33]:
# Other things to import
import datetime

In [5]:
# If this complains about SHORTCUT_API_TOKEN, check out the README.md for instructions
client = ShortcutClient().validate()

In [8]:
# Who am I?
member = client.get_json('/member')
member_id = member['id'] # for later
member

{'id': '66105996-2f33-4dee-8066-3e9d7b8098d9',
 'name': 'Daniel | Gregoire',
 'mention_name': 'danielgregoire7246',
 'workspace2': {'url_slug': 'shortcut-pivotal-import',
  'estimate_scale': [0, 1, 2, 4, 8]}}

In [9]:
# What workspace am I in?
client.get_json('/member')['workspace2']['url_slug']

'shortcut-pivotal-import'

In [11]:
# What stories do I currently own?
my_stories = client.post_json('/stories/search', {'owner_ids': [member_id]})
len(my_stories)

1219

In [38]:
# How many are in an in-progress state?
len([s for s in my_stories if s['started'] and not s['completed']])

717

In [36]:
# How many were started more than N days ago?
ndays = 7
ago = datetime.datetime.now(datetime.UTC) - timedelta(days=ndays)
started = [datetime.datetime.fromisoformat(s['started_at']) for s in my_stories if s['started']]
len([dt for dt in started if dt < ago])

1219

_NB: If you're interested in more analytical use-cases for Shortcut's API, you should review the [Analysis.ipynb](Analysis.ipynb) Jupyter notebook in the repo._

## Members & Groups

Shortcut calls users "members" and the v3 API calls teams "groups".

Entities include references to the ids of members and groups in various places. If you need to see the full names or mention names of users and groups, you need to pull down `/members` and `/groups` ahead of time and create a mapping.

Here's an example of creating a Python dict mapping member IDs to their mention names.

In [40]:
members = client.get_json('/members')
{m['id']:m['profile']['mention_name'] for m in members}

{'66105996-2f33-4dee-8066-3e9d7b8098d9': 'danielgregoire7246',
 '66105b5e-ec97-477e-b6ef-19c15132442c': 'daniel.l.gregoire+clubhousedanieltestlabs-1',
 '66105b5f-c478-4d56-ac90-9abe6d7ede1b': 'chrisdemwell',
 '66105b5f-07e9-4694-950a-65c665e1d867': 'osei.poku',
 '6615b0f7-0c5e-447a-a115-bdba38f44d54': 'daniel.gregoire+unknown',
 '66a2948b-4dd2-4e20-a550-671e2326e508': 'santiago1979',
 '66a3f20b-258e-4bb8-96ca-b9871a8bee1f': 'german0096',
 '66a7aa63-4b46-4cde-be6a-708336d7cfe5': 'aaguirre',
 '66aa78e4-700b-44de-996a-d1d86c95bf36': 'aaguirre+2',
 '66b4e5a9-81c9-41e2-a316-d7cffa0d871a': 'smunoz+1',
 '66ba7ac1-a7b0-4d8f-a5b2-24389a53cdd1': 'watash1a',
 '66c8c384-026e-4d2c-9749-a2ce7ea8cca7': 'bcfadmin1',
 '66cf541c-0ed9-4156-ad7a-734280685f7d': 'malcom+admin',
 '66cf59d3-07ef-4325-add6-4f806f2fdf59': 'mal9437',
 '66d0b7d9-c09b-4a5e-bcfc-0aaecbbce310': 'bcfpivotaladmin',
 '67587ac1-5d38-4b2e-9f8f-f7ba09d13dfe': 'mike_thorpe'}

## Workflows and Workflow States

Shortcut Stories are always in one Workflow State. Workflow States represents points in your project management process and are bucketed into backlog, unstarted, started, and done types.

Every Workflow State has a unique ID that identifies it. Every Workflow State is also situated within a Workflow. If you're using a paid Shortcut plan, you're able to model how different teams at your company work by having distinct Workflows and constituent Workflow States.

As with members and groups, Shortcut API payloads that refer to Workflow States do so with an ID. If you need to see the name of the Workflow State and/or Workflow, you'll need to request `/workflows` ahead of time and create a mapping.

Here's an example of creating a mapping from Workflow State IDs to their names:

In [52]:
workflows = client.get_json('/workflows')
workflow_states_by_id = {}
for w in workflows:
    for s in w['states']:
        workflow_states_by_id[s['id']] = s['name']
workflow_states_by_id

{500000006: 'Backlog',
 500000007: 'Unstarted',
 500000008: 'In Progress',
 500000009: 'In Review',
 500000010: 'Done'}

If in your scripts you need to distinguish workflows or workflow state types, make your value in this dict more complex and add entries for the things you need.

## Searching

Shortcut's v3 REST API features two different endpoints for searching:

- [Search Stories (Old)](https://developer.shortcut.com/api/rest/v3#Search-Stories-Old)
- [Search](https://developer.shortcut.com/api/rest/v3#Search)

The former provides a structured interface for querying story data in your Shortcut workspace. The second targets Shortcut's full-text search, operators for which are [documented in Shorcut's help center](https://help.shortcut.com/hc/en-us/articles/360000046646-Searching-in-Shortcut-Using-Search-Operators) and which searches across stories, epics, iterations, and objectives.

## Bulk Operations

Shortcut's API allows performing bulk operations that are otherwise cumbersome to perform in the UI itself.

The [Stories-specific endpoints](https://developer.shortcut.com/api/rest/v3#Stories) support first-class bulk operations for creating, updating, and deleting Shortcut Stories. Use these rather than individual API calls for faster performance and to avoid hitting the rate limit.

## Python Tips

### What keys are in a given Shortcut entity?

A sorted list of keys can be helpful for discovering or remembering what's available in a given Shortcut entity. When deserialized from JSON, these are represented as Python dicts.

In [54]:
my_dict = {'a':'alpha', 'g':'gamma', 'b':'beta'}
sorted(my_dict.keys())

['a', 'b', 'g']